In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

# data

In [ ]:
# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=9)
databunch = MultiDeptTabularDataBunch.from_df('tmp', context.cyc_cont, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

In [ ]:
# read context and append 
geo_raw = pd.read_csv('/home/wenxiang/datasets/tbm/geo_csv/geo2.1.csv', index_col=0)
geo_cols = ['起始桩号', '最后桩号', '实际开挖围岩类别', '洞室部位主要岩性']
geo = geo_raw.loc[:,geo_cols]
# clean nan
geo = geo.loc[geo.iloc[:,-1].notna()]
zh = context.cyc_cont.iloc[:,-5]
cat_names = geo.columns.tolist()

idx_dep_var3 = []
missing_idx = -1 #geo.shape[0] # last item + 1
for z in list(zh):
    is_within = ((z - geo.iloc[:,0]) * (z - geo.iloc[:,1])) < 0
    if is_within.any():
        idx_dep_var3.append(np.argmax(is_within.values))
    else:
        idx_dep_var3.append(missing_idx)

dep_var3 = []
for i in idx_dep_var3:
    dep_var3.append(geo.iloc[i,2:].tolist() if i != missing_idx else ['oor', 'oor'])

cycles = pd.concat([context.cyc_cont, pd.DataFrame(dep_var3, columns=['围岩等级', '岩性'])], axis=1)
cat_names = cycles.columns[-2:]
cont_names = cycles.columns[:-2]

# simplify
cat_1 = cycles.loc[:,cat_names[0]]
III_idx = np.logical_or(cat_1 == 'Ⅲa', cat_1 == 'Ⅲb')
cycles.loc[III_idx,cat_names[0]] = 'Ⅲ'
IV_idx = np.logical_or(cat_1 == 'Ⅳ好', cat_1 == 'Ⅳ差')
cycles.loc[IV_idx,cat_names[0]] = 'Ⅳ'

cf = Categorify(cat_names, cont_names)
cf.apply_train(cycles)

fn_feather = Path(context.fn_feather)
new_fn_feather = fn_feather.parent / (fn_feather.stem + '-cat' + fn_feather.suffix)
cycles.to_feather(new_fn_feather)

# run

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *

In [ ]:
n_cont = 2
n_cat = [5, 6]
cat_names = ['围岩等级', '岩性']

# context = Context('amended-1-no_noise', debug=0, dep_var=dep_var2, mulr=1, gpu_start=9)
context = Context('amended-3-7-cat', debug=0, dep_var=dep_var2 + cat_names, mulr=7, gpu_start=9)
cycles = context.cyc_cont

# un-categorify
cat_columns = []
for cat_name in cat_names:
    cat_columns.append(cycles.loc[:,cat_name])
    cycles.loc[:,cat_name] = cycles.loc[:,cat_name].cat.codes.values
    
databunch = MultiDeptTabularDataBunch.from_df('tmp', cycles, context.dep_var, valid_idx=context.valid_idx_tile, bs=context.bs)

# 模型验证

In [ ]:
hyper_params = {
    'use_extra_x': 1,
    'n_hidden': 1000,
    'n_layers': 3,
    'hidden_p': 0.075,
    'input_p': 0.1,
    'weight_p': 0.125,
    'drops': [0.1] * 4,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
    'loss_func': DualLoss(n_cont, n_cat, 0.1)
}

hyper_params['layers'] = [3*(context.n_cont) + int(hyper_params['use_extra_x']) * 3, 1000, 13]
learner = get_new_model(context, databunch, hyper_params)

# learner.fit_one_cycle(240, max_lr=1e-1)

In [ ]:
lr = 1e-2
time_str = time.strftime("%Y-%m-%d_%H:%M:%S")
save_name = '_'.join(['final-no-extra-day', str(hyper_params['n_hidden']),context.exp_name,time_str])
callbacks = [SaveModelCallback(learner, name=save_name , every='improvement', monitor='MAPD', mode='min')]
learner.fit_one_cycle(240, lr, callbacks=callbacks)
learner.recorder.plot_metrics()

In [ ]:
lr = 1e-2
time_str = time.strftime("%Y-%m-%d_%H:%M:%S")
save_name = '_'.join(['final-no-extra-day', str(hyper_params['n_hidden']),context.exp_name,time_str])
callbacks = [SaveModelCallback(learner, name=save_name , every='improvement', monitor='MAPD', mode='min')]
learner.fit_one_cycle(240, lr, callbacks=callbacks)
learner.recorder.plot_metrics()

In [ ]:
xb_valid, yb_valid = ni(databunch.valid_dl)
yb_valid = to_np(yb_valid)

In [ ]:
yb_cont_valid_denorm = denormalize(yb_valid[:,:2], *context.stat_y)

In [ ]:
learner.load('final-no-extra-day_1000_amended-3-7-cat_2019-04-29_22:18:11', purge=False)
model = learner.model
model.eval()
model.reset()
pb_valid = model(*xb_valid)[0]

# pb = denormalize(pb[0], *context.stat_y)
# pb = pb[0]
# pb = to_np(pb)
# pb_valid = pb_valid[0]


In [ ]:
p_cont, p_cat = pb_valid[:,:n_cont], pb_valid[:,n_cont:]

cat_idx_zip = [0] + hyper_params['loss_func'].cat_idx.tolist()
p_cat_tasks = []
for i,j in zip(cat_idx_zip[:-1], cat_idx_zip[1:]):
    p_cat_tasks.append(to_np(p_cat[:,i:j]))

p1, p2 = [o.argmax(1) for o in p_cat_tasks]

In [ ]:
pb_valid = denormalize(p_cont, *context.stat_y)
pb_valid = to_np(pb_valid)

In [ ]:
# mre
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

i = 0
print('mre', (np.abs(pb_valid[:,i] - yb_cont_valid_denorm[:,i]) / yb_cont_valid_denorm[:,i]).mean())
print('rmse', sqrt(mean_squared_error(pb_valid[:,i], yb_cont_valid_denorm[:,i])))
print('r2_score', r2_score(pb_valid[:,i], yb_cont_valid_denorm[:,i]))

In [ ]:
# mre
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

i = 1
print('mre', (np.abs(pb_valid[:,i] - yb_cont_valid_denorm[:,i]) / yb_cont_valid_denorm[:,i]).mean())
print('rmse', sqrt(mean_squared_error(pb_valid[:,i], yb_cont_valid_denorm[:,i])))
print('r2_score', r2_score(pb_valid[:,i], yb_cont_valid_denorm[:,i]))

In [ ]:
# Now let's quickly train a random forest with various kinds of datasets
from sklearn.ensemble import forest
from sklearn.ensemble import RandomForestRegressor
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)
    
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
xb, yb = ni(databunch.train_dl)
X_train, y_train = to_np(xb[1]), to_np(yb)[:,:2]

xb, yb = ni(databunch.valid_dl)
X_valid, y_valid = to_np(xb[1]), to_np(yb)[:,:2]
# m = RandomForestRegressor(n_jobs=-1, n_estimators=80, oob_score=True, min_samples_leaf=5)
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, oob_score=True, min_samples_leaf=5)
m.fit(X_train, y_train)

p_train, p_valid = m.predict(X_train), m.predict(X_valid)
# random forest
df_scores = pd.DataFrame({
    'Train MAPD': [context.metrics(torch.tensor(p_train, dtype=torch.float), torch.tensor(y_train, dtype=torch.float)).item()],
    'Valid MAPD': [context.metrics(torch.tensor(p_valid, dtype=torch.float), torch.tensor(y_valid, dtype=torch.float)).item()],
    'Train score': [m.score(X_train, y_train)],
    'Valid score': [m.score(X_valid, y_valid)],
    'OOB': [m.oob_score_],
})
df_scores

In [ ]:
pb_valid = denormalize(p_valid, *context.stat_y)
yb_cont_valid_denorm = denormalize(y_valid, *context.stat_y)

In [ ]:
i = 0
print('mre', (np.abs(pb_valid[:,i] - yb_cont_valid_denorm[:,i]) / yb_cont_valid_denorm[:,i]).mean())
print('rmse', sqrt(mean_squared_error(pb_valid[:,i], yb_cont_valid_denorm[:,i])))
print('r2_score', r2_score(pb_valid[:,i], yb_cont_valid_denorm[:,i]))

In [ ]:
i = 1
print('mre', (np.abs(pb_valid[:,i] - yb_cont_valid_denorm[:,i]) / yb_cont_valid_denorm[:,i]).mean())
print('rmse', sqrt(mean_squared_error(pb_valid[:,i], yb_cont_valid_denorm[:,i])))
print('r2_score', r2_score(pb_valid[:,i], yb_cont_valid_denorm[:,i]))

In [ ]:
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
class_names = cat_columns[0].cat.categories

y_test = yb_valid[:,-2].astype(np.int)
y_pred = p1
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='未正规化的岩体分类混淆矩阵')
set_ax_font(plt.gca())
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='正规化的岩体分类混淆矩阵')
set_ax_font(plt.gca())

In [ ]:
class_names = cat_columns[1].cat.categories
y_test = yb_valid[:,-1].astype(np.int)
y_pred = p2
plot_confusion_matrix(y_test, y_pred, classes=class_names,
                      title='未正规化的岩性分类混淆矩阵')
set_ax_font(plt.gca())
plot_confusion_matrix(y_test, y_pred, classes=class_names, normalize=True,
                      title='正规化的岩性分类混淆矩阵')
set_ax_font(plt.gca())

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.utils.fixes import signature
    
def eval_mult_class(Y_test, y_score):
    # For each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                            y_score[:, i])
        average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
        y_score.ravel())
    average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                         average="micro")
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'
          .format(average_precision["micro"]))
    return precision, recall, average_precision


def plot_mult_class(precision, recall, average_precision):
    plt.figure()
    plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2,
             where='post')
    step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
    plt.fill_between(recall["micro"], precision["micro"], alpha=0.2, color='b',
                     **step_kwargs)

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title(
        'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
        .format(average_precision["micro"]))

In [ ]:
# p2
y_test = yb_valid[:,-1].astype(np.int)
y_pred = p2

classes = np.unique(np.array(y_test))
Y_test, y_score = (label_binarize(o, classes=classes) for o in (y_test, y_pred))
n_classes = len(classes)
# Y_test, y_score = y_test, y_pred

precision, recall, average_precision = eval_mult_class(Y_test, y_score)
plot_mult_class(precision, recall, average_precision)

In [ ]:
# p1
y_test = yb_valid[:,-2].astype(np.int)
y_pred = p1

classes = np.unique(np.array(y_test))
Y_test, y_score = (label_binarize(o, classes=classes) for o in (y_test, y_pred))
n_classes = len(classes)
# Y_test, y_score = y_test, y_pred

precision, recall, average_precision = eval_mult_class(Y_test, y_score)
plot_mult_class(precision, recall, average_precision)